# Partial Dependence Plot Generator

## Detailed Analysis

This script is used to interpret the decision-making logic of the trained machine learning models. It generates Partial Dependence Plots (PDPs), which visualize how the predicted probability of a star being a "Quark Star" changes as a single feature is varied, while the effects of all other features are averaged out.

The script focuses on two distinct models:
1.  **Model A:** To understand how macroscopic observables (Mass, Radius, Tidal Deformability) influence the classification.
2.  **Model D:** To understand how microscopic properties (Density, Sound Speed, Topology) drive the model's decisions.

## Physics and Math

### Partial Dependence Function
The machine learning models approximate a function $f(X)$ that outputs the probability of the positive class (Quark). To understand the marginal effect of a specific feature $x_S$ (e.g., Radius) independent of the other features $X_C$, the partial dependence function is computed.

Mathematically, the partial dependence function $\hat{f}_{x_S}$ is estimated by averaging the model predictions over the distribution of the other features in the dataset:

$$
\hat{f}_{x_S}(x_S) = \frac{1}{N} \sum_{i=1}^{N} f(x_S, x_{C}^{(i)})
$$

where $\{x_{C}^{(1)}, \dots, x_{C}^{(N)}\}$ are the values of the complementary features observed in the training data.

*   **Y-Axis:** Represents the average predicted probability $P(\text{Quark})$.
*   **X-Axis:** Represents the range of values for the feature of interest.

This technique reveals whether the model has learned physically meaningful relationships (e.g., "smaller radii imply a higher probability of being a Quark star") or if it is relying on artifacts.

## Code Walkthrough

### 1. Configuration
Common settings for the PDP calculation are defined, such as the grid resolution (number of points on the x-axis) and the method (`kind='average'`).

```python
common_params = {
    "kind": "average",      
    "grid_resolution": 50,  
    "percentiles": (0.01, 0.99),
    "n_jobs": -1
}
```

### 2. Plot 1: Observables (Model A)
The script first checks if `Model A` is present in the input dictionary.
*   **Feature Selection:** The features `Mass`, `Radius`, and `LogLambda` are selected.
*   **Computation:** `PartialDependenceDisplay.from_estimator` computes the curves efficiently using the tree structure of the Random Forest.
*   **Formatting:** Axis limits are applied from `CONSTANTS`. A horizontal line at $P=0.5$ marks the decision boundary.

### 3. Plot 2: Microphysics (Model D)
The script then processes `Model D`, which was trained on internal physics features.
*   **Feature Selection:** `Eps_Central` (Density), `CS2_Central` (Sound Speed), and `Slope14` (Topology) are analyzed.
*   **Shading:** For the "Slope" panel, background shading is added to highlight physical regimes:
    *   **Red Region (Slope < 0):** Hadronic-like behavior (gravitational compression dominates).
    *   **Green Region (Slope > 0):** Quark-like behavior (vacuum pressure stabilizes the star against gravity).

```python
# Left of 0.0: Hadronic-like (Collapsing branch)
axes[2].axvspan(CONSTANTS['PLOT_SLOPE_LIM'][0], 0.0, color=COLORS['H_main'], alpha=0.1)
# Right of 0.0: Quark-like (Stable branch)
axes[2].axvspan(0.0, CONSTANTS['PLOT_SLOPE_LIM'][1], color=COLORS['Q_main'], alpha=0.1)
```

## Visualization Output

The script saves two PDF files to the `plots/` directory.

### 1. `fig_pdp_observables.pdf`
*   **Panels:** Mass, Radius, Log Tidal Deformability.
*   **Y-Axis:** Probability $P(\text{Quark})$.
*   **Interpretation:**
    *   **Radius:** Typically shows a sharp increase in quark probability as radius decreases below $\sim 11-12$ km.
    *   **Tidal:** Shows how probability varies with deformability (Quark stars are often less deformable).

### 2. `fig_pdp_microphysics.pdf`
*   **Panels:** Central Density, Sound Speed, Slope.
*   **Y-Axis:** Probability $P(\text{Quark})$.
*   **Interpretation:**
    *   **Density:** Higher central densities generally correlate with higher quark probability.
    *   **Sound Speed:** Shows if the model associates high stiffness ($c_s^2 > 1/3$) with quark matter.
    *   **Slope:** Demonstrates the phase transition. The probability curve usually transitions from 0 to 1 as the slope crosses from negative to positive, confirming that the model utilizes the topological stability criterion.